In [1]:
import psycopg2
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# get taxi initial points
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

initial_points = []
sql = '''
select st_transform(initial_point, 3763)
from taxi_services 
'''
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

for row in results:
    initial_points.append(row[0])
print(initial_points[:5])

['0101000020B30E0000DCCDA3092D31E3C09F8BE867904B0441', '0101000020B30E00002877A4E9642FE2C0445BEC854A020441', '0101000020B30E00007D59F7E82972E3C0B1967DA256090441', '0101000020B30E000074BC0AE11660E4C07BDDB2DBD7290441', '0101000020B30E00007C232BBD37B0E3C025F7A7F230270441']


In [3]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# get the closest bus stop for each initial point
initial_stops = []
for initial_point in initial_points[:5]:
    sql = f'''
    select 
        stop_id,
        st_transform(t.initial_point, 3763) as ip,
        st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763),
        st_transform(t.final_point, 3763),
        t.initial_ts
    from taxi_services t
    cross join stops s
    where st_transform(t.initial_point, 3763) = '{initial_point}'
    order by st_distance(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763), '{initial_point}')
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    for row in results:
        initial_stops.append((row[0], row[2], row[3], row[4]))
print(initial_stops[:5])

[('COMB2', '0101000020B30E00008683A8946A29E3C03192DC5FB84B0441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('COMB1', '0101000020B30E0000F656F70DDF25E3C050971C19D54B0441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('PSOC1', '0101000020B30E0000F18341A7953AE3C0DDEE33D0A1460441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('LIMA1', '0101000020B30E0000F89AD339DA1BE3C023B4EA3CE8470441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('LIMA2', '0101000020B30E0000B703E7B0161FE3C0EF8E9D1D77460441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621)]


In [4]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

stop_ids = []

# todos os autocarros que param nessa initial_stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    select trip_id from stop_times where stop_id = '{initial_stop[0]}'
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    stop_ids.append(results[0])
print(stop_ids[:5])

[('5M_1_D_1',), ('5M_0_D_8',), ('701_0_D_11',), ('806_0_D_15',), ('806_1_D_6',)]


In [20]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# now we get the final stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    with t2 as ( SELECT DISTINCT ON (t.route_id) t.route_id, st.trip_id, st.departure_time, '{datetime.fromtimestamp(initial_stop[3]).time()}', st.stop_id
                    FROM stop_times st
                    JOIN trips t ON st.trip_id = t.trip_id
                    WHERE st.stop_id = '{initial_stop[0]}'
                      AND st.departure_time > '{datetime.fromtimestamp(initial_stop[3]).time()}'
                    ORDER BY t.route_id, extract(epoch from (st.departure_time - '{datetime.fromtimestamp(initial_stop[3]).time()}')))

    select st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}') as p,
    t2.route_id, 
    t2.trip_id,
    s.stop_id
    from t2 
    join stops s on s.stop_id = t2.stop_id
    group by t2.route_id, t2.trip_id, s.stop_id
    order by st_distance(st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}'), '{initial_stop[2]}')
    limit 1
    '''

    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    print(results, "\n")

[('0101000020B30E00008683A8946A29E3C03192DC5FB84B0441', '5M', '5M_1_D_1', 'COMB2')] 

[('0101000020B30E0000F656F70DDF25E3C050971C19D54B0441', '5M', '5M_0_D_2', 'COMB1')] 

[('0101000020B30E0000F18341A7953AE3C0DDEE33D0A1460441', '5M', '5M_0_D_2', 'PSOC1')] 

[('0101000020B30E0000F89AD339DA1BE3C023B4EA3CE8470441', '806', '806_0_U_19', 'LIMA1')] 

[('0101000020B30E0000B703E7B0161FE3C0EF8E9D1D77460441', '806', '806_1_S_1', 'LIMA2')] 

[('0101000020B30E00004CE3657D4633E3C0B225F9E870520441', '204', '204_1_U_113', 'ACST2')] 

[('0101000020B30E000034A182BD2D4AE3C018380C77814E0441', '204', '204_1_U_113', 'BJ1')] 

[('0101000020B30E0000D369B4374A50E3C039BC3A56E64E0441', '204', '204_0_D_1', 'BJ3')] 

[('0101000020B30E0000612ADAAABA0DE3C0554D5051404E0441', '806', '806_0_U_19', 'COMB3')] 

[('0101000020B30E00009B4D46850F48E3C0D29E9A020D440441', '5M', '5M_1_D_1', 'PSOC2')] 

[('0101000020B30E00004A94150D220DE3C0BF224527D44E0441', '806', '806_1_S_1', 'COMB4')] 

[('0101000020B30E000032F4068C3643E3C08